# Analyzing the data

In [397]:
# import pandas as pd
# import numpy as np

# # data 5 is bad and 9 is largest

# df = pd.read_csv("odds_datas/test_9_odds_data.csv") # odds_data
# df["T_goals"] = df["H_score"] + df["A_score"]
# df

# STRATEGIES
### Overall daily target should be less or equal to plus 50% of initial target
example:
<br>intial target = 100
<br>overall target = 150 or less

### All Odds Data

In [54]:
import pandas as pd
import numpy as np

# data 5 is baddest and 9 is largest

df = pd.read_csv("odds_datas/test_2_odds_data.csv") # odds_data

check_list = []
odd_list = []
for _, row in df.iterrows():
    check_list.append((row["week"], len(check_list)+1, row["over_3"], row["H_score"], row["A_score"]))
    if len(check_list) == 10:
        # print(check_list)
        sorted_odds = sorted(check_list, key=lambda x: x[2], reverse=True)
        all_odds = [x[2] for x in sorted_odds]
        for odd in sorted_odds:
            if all_odds.count(odd[2]) == 1 and odd[2] != max(all_odds):
                odd_list.append(odd)
                break               
        check_list = []

new_df = pd.DataFrame(odd_list, columns=["week", "odd_position_num", "odd", "H_score", "A_score"])
# new_df = pd.read_csv("auto_trade_records/rec_2.csv")

print("Total Stake: ", len(new_df))

print("Mean Odd:", np.mean(new_df["odd"]))
new_df["T_goals"] = new_df["H_score"] + new_df["A_score"]
# new_df = new_df[new_df["odd"] >= 3.40]
new_df

Total Stake:  319
Mean Odd: 3.5550470219435737


,week,odd_position_num,odd,H_score,A_score,T_goals
0,1,3,3.41,2,2,4
1,2,8,3.58,2,3,5
2,3,9,3.53,4,2,6
3,4,2,3.48,0,0,0
4,5,6,3.45,0,0,0
...,...,...,...,...,...,...
314,34,4,3.30,0,0,0
315,35,8,3.43,1,0,1
316,36,9,3.40,4,1,5
317,37,7,3.48,0,0,0


# Single Stake

In [70]:
bankroll = 100000
highest_loss_prtg = 5
highest_loss = -(int((highest_loss_prtg * bankroll) / 100))
print("Highest Loss:", highest_loss)

single_stake_prtg = 1
single_stake = int((single_stake_prtg * bankroll) / 100)
print("Single Target -", single_stake)

total_target_prtg = 2
total_target = int((total_target_prtg * bankroll) / 100)
print("Total Target -", total_target, "\n")

roll_over_times = 3
total_bankroll = 0
restart = False
bank_roll_list = []
loss_count = 0
df_new = new_df[["odd", "T_goals"]].copy()
for i, row in df_new.iterrows():
    if restart == True:
        total_bankroll = 0
    if row["T_goals"] > 3:
        if loss_count < roll_over_times:
            total_bankroll += (row["odd"] - 1) * single_stake
            if total_bankroll >= total_target:
                restart = True
            else:
                restart = False
        loss_count = 0
    else:
        restart = False
        if loss_count < roll_over_times:
            total_bankroll -= single_stake
            if total_bankroll <= highest_loss:
                restart = True
                loss_count += roll_over_times + 1
        loss_count += 1
    bank_roll_list.append(total_bankroll)


df_new["Single_BR"] = bank_roll_list
df_new["Target"] = np.where(df_new["Single_BR"] >= total_target, "Achieved", "-")

counter = 1
counter_list = []

for target in df_new['Target']:
    counter_list.append(counter)
    counter += 1
    if target == 'Achieved':
        counter = 1

df_new['counter'] = counter_list
df_new

Highest Loss: -5000
Single Target - 1000
Total Target - 2000 



,odd,T_goals,Single_BR,Target,counter
0,3.41,4,2410.0,Achieved,1
1,3.58,5,2580.0,Achieved,1
2,3.53,6,2530.0,Achieved,1
3,3.48,0,-1000.0,-,1
4,3.45,0,-2000.0,-,2
...,...,...,...,...,...
314,3.30,0,-240.0,-,4
315,3.43,1,-1240.0,-,5
316,3.40,5,1160.0,-,6
317,3.48,0,160.0,-,7


In [71]:
targe_count = df_new[df_new["Target"] == "Achieved"]
print("Total Target Won:", len(targe_count))

print("Sum Profits:", targe_count["Single_BR"].sum())
targe_count

Total Target Won: 15
Sum Profits: 38780.0


,odd,T_goals,Single_BR,Target,counter
0,3.41,4,2410.0,Achieved,1
1,3.58,5,2580.0,Achieved,1
2,3.53,6,2530.0,Achieved,1
35,3.45,6,2450.0,Achieved,33
50,3.76,5,2760.0,Achieved,15
127,3.76,5,3000.0,Achieved,77
203,3.42,5,2420.0,Achieved,76
224,3.76,4,3120.0,Achieved,21
225,3.29,6,2290.0,Achieved,1
255,3.58,5,2580.0,Achieved,30


In [72]:
losses = df_new[df_new["Single_BR"] <= highest_loss]
print("All Highest Losses:", len(losses))
print("Sum Losses:", losses["Single_BR"].sum())

highest_loss_amount = df_new["Single_BR"].min()
print("Highest Loss Amount:", highest_loss_amount)

losses

All Highest Losses: 11
Sum Losses: -59190.0
Highest Loss Amount: -5880.0


,odd,T_goals,Single_BR,Target,counter
31,3.43,2,-5170.0,-,29
45,3.58,1,-5000.0,-,10
60,3.45,2,-5710.0,-,10
79,3.43,1,-5570.0,-,29
102,3.58,2,-5580.0,-,52
166,3.76,2,-5800.0,-,39
198,3.42,0,-5880.0,-,71
234,3.76,2,-5000.0,-,9
253,3.76,3,-5240.0,-,28
262,3.29,2,-5000.0,-,6


# Martingale Stake

In [35]:
bankroll = 100000
loss_amount = 3700
highest_loss_prtg = 5
highest_loss = -(int((highest_loss_prtg * bankroll) / 100))
print("Highest Loss:", highest_loss)

single_target_prtg = 1
single_target = int((single_target_prtg * bankroll) / 100)
print("Single Target -", single_target)

# single_stake_prtg = 1
# single_stake = int((single_stake_prtg * bankroll) / 100)
# print("Single Target -", single_stake)

total_target_prtg = 3
total_target = int((total_target_prtg * bankroll) / 100)
print("Total Target -", total_target, "\n")

roll_over_times = 5
total_bankroll = 0
restart = False
bank_roll_list = []
loss_count = 0

df_new_2 = new_df[["odd", "T_goals"]].copy()

for i, row in df_new_2.iterrows():
    if restart == True:
        total_bankroll = 0
    if row["T_goals"] > 3:
        if loss_count < roll_over_times:
            total_bankroll += single_target
            if total_bankroll >= total_target:
                restart = True
            else:
                restart = False
        loss_count = 0
    else:
        loss_count += 1
        restart = False
        if loss_count == roll_over_times:
            total_bankroll -= loss_amount
            if total_bankroll <= highest_loss:
                restart = True
                loss_count += roll_over_times + 1
        
    bank_roll_list.append(total_bankroll)


df_new_2["Martingale_BR"] = bank_roll_list
df_new_2["Target"] = np.where(df_new_2["Martingale_BR"] >= total_target, "Achieved", "-")

counter = 1
counter_list = []
for target in df_new_2['Target']:
    counter_list.append(counter)
    counter += 1
    if target == 'Achieved':
        counter = 1

df_new_2['counter'] = counter_list
df_new_2

Highest Loss: -5000
Single Target - 1000
Total Target - 3000 



,odd,T_goals,Martingale_BR,Target,counter
0,3.43,2,0,-,1
1,3.76,2,0,-,2
2,3.29,2,0,-,3
3,3.21,3,0,-,4
4,3.83,3,-3700,-,5
...,...,...,...,...,...
641,3.53,2,-1700,-,20
642,3.58,2,-1700,-,21
643,3.45,3,-5400,-,22
644,3.76,3,0,-,23


In [36]:
targe_count = df_new_2[df_new_2["Target"] == "Achieved"]
print("Total Target Won:", len(targe_count))

print("Sum Profits:", targe_count["Martingale_BR"].sum())
targe_count

Total Target Won: 25
Sum Profits: 77400


,odd,T_goals,Martingale_BR,Target,counter
23,3.45,6,3000,Achieved,24
30,3.83,5,3000,Achieved,7
55,3.76,6,3300,Achieved,25
115,3.30,5,3000,Achieved,60
123,3.76,4,3000,Achieved,8
130,3.53,6,3000,Achieved,7
150,3.29,5,3000,Achieved,20
155,3.76,4,3000,Achieved,5
164,3.45,5,3000,Achieved,9
192,3.76,4,3300,Achieved,28


In [37]:
losses = df_new_2[df_new_2["Martingale_BR"] <= highest_loss]
print("All Highest Losses:", len(losses))
print("Sum Losses:", losses["Martingale_BR"].sum())

highest_loss_amount = df_new_2["Martingale_BR"].min()
print("Highest Loss Amount:", highest_loss_amount)
losses

All Highest Losses: 9
Sum Losses: -59500
Highest Loss Amount: -8200


,odd,T_goals,Martingale_BR,Target,counter
10,3.76,2,-7400,-,11
68,3.76,2,-6400,-,13
98,3.45,3,-6100,-,43
144,3.53,2,-6400,-,14
240,3.76,1,-6400,-,18
438,3.42,2,-8200,-,87
505,3.76,1,-7800,-,55
532,3.53,1,-5400,-,18
643,3.45,3,-5400,-,22


In [99]:
df_new_2[:50]

,odd,T_goals,Martingale_BR,Target,counter
0,3.88,1,0,-,1
1,3.83,3,0,-,2
2,3.76,4,1000,-,3
3,3.88,1,1000,-,4
4,3.29,5,2000,-,5
5,3.85,3,2000,-,6
6,3.76,6,3000,Achieved,7
7,3.88,2,0,-,1
8,3.43,4,1000,-,2
9,3.85,3,1000,-,3


In [100]:
df_new_2[50:100]

,odd,T_goals,Martingale_BR,Target,counter
50,3.76,3,-6100,-,44
51,3.40,3,-6100,-,45
52,3.85,1,-9800,-,46
53,3.85,3,-9800,-,47
54,3.76,0,-9800,-,48
55,3.76,1,-9800,-,49
56,3.88,3,-9800,-,50
57,3.42,4,-9800,-,51
58,3.85,3,-9800,-,52
59,3.42,0,-9800,-,53


In [101]:
df_new_2[100:150]

,odd,T_goals,Martingale_BR,Target,counter
100,3.76,1,-2500,-,94
101,3.88,2,-2500,-,95
102,3.41,2,-2500,-,96
103,3.83,3,-2500,-,97
104,3.43,4,-1500,-,98
105,3.43,3,-1500,-,99
106,3.32,2,-1500,-,100
107,3.53,0,-1500,-,101
108,3.76,1,-1500,-,102
109,3.45,2,-5200,-,103


In [90]:
df_new_2[150:200]

,odd,T_goals,Single_BR,Target,counter


In [ ]:
df_new_2[200:250]

In [ ]:
df_new_2[250:300]

In [ ]:
df_new_2[300:350]

In [423]:
count = 1
martingale_list = []
for x in new_df["T_goals"]:
    if x > 3:
        # print(count)
        martingale_list.append(count)
        count = 1             
    else:
        count += 1

print("Total Wins:", len(martingale_list))
print(martingale_list)
print(sorted(martingale_list, reverse=True)[:100])
print("Hightest Martingale Level:", max(martingale_list))

Total Wins: 21
[2, 1, 2, 3, 2, 1, 1, 9, 1, 3, 1, 1, 5, 2, 4, 4, 2, 3, 1, 1, 7]
[9, 7, 5, 4, 4, 3, 3, 3, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1]
Hightest Martingale Level: 9


# New over_3 Data

### Martingale vs Single Stake Strategy

In [30]:
import pandas as pd
import numpy as np

df = new_df.copy()
# df = pd.read_csv("auto_trade.csv")
df["T_goals"] = df["H_score"] + df["A_score"] 
print("TOTAL GAMES:", len(df), "\n")

# Split the rows
split_size = 38
array = df.copy().to_numpy()
split_arrays = np.array_split(array, range(split_size, len(array), split_size))

# Convert the split arrays back to DataFrames
split_dfs = [pd.DataFrame(arr, columns=df.columns) for arr in split_arrays]

# Access the split DataFrames
for odd_df in split_dfs:
    # print(split_df)

    bankroll = 100000
    loss_amount = 3700
    highest_loss = -(loss_amount * 3)
    print("Highest Loss :", highest_loss)

    mart_single_target_prtg = 1
    mart_single_target = (mart_single_target_prtg * bankroll) / 100
    print("Single Target -", mart_single_target)

    total_target_prtg = 3
    total_target = (total_target_prtg * bankroll) / 100
    print("Total Target -", total_target, "\n")


    single_bankroll = 0
    single_stake = 1000
    single_bankroll_list = []

    martingale_bankroll = 0
    total_loss = 0
    martingale_bankroll_list = []
    mart_counter = 0
    single_counter = 0

    for _, row in odd_df.iterrows():
        if row["T_goals"] > 3:
            if single_counter <= 5:
                single_bankroll += (row["odd"] - 1) * single_stake
            single_counter = 0
            if mart_counter <= 5:
                martingale_bankroll += mart_single_target 
            mart_counter = 0
        else:
            single_counter += 1
            if single_counter <= 5:
                single_bankroll -= single_stake
            mart_counter += 1
            if mart_counter == 5:
                martingale_bankroll -= loss_amount
        martingale_bankroll_list.append(int(martingale_bankroll))
        single_bankroll_list.append(int(single_bankroll))

    odd_df["Single_Bankroll"] = single_bankroll_list
    odd_df["Check_single"] = np.where(odd_df["Single_Bankroll"] >= total_target, "Achieved", "-")

    odd_df["Martingale_Bankroll"] = martingale_bankroll_list    
    odd_df["Check_Martingale"] = np.where(odd_df["Martingale_Bankroll"] >= total_target, "Achieved", "-")   
    print("TOTAL GAMES:", len(odd_df))


    mart_targets = odd_df[odd_df["Check_Martingale"] == "Achieved"]
    single_targets = odd_df[odd_df["Check_single"] == "Achieved"]

    mart_single_list = [mart_targets, single_targets]

    for temp in range(len(mart_single_list)):
        val = "Martingale_Bankroll" if temp == 0 else "Single_Bankroll"
        if len(mart_single_list[temp]) > 0:
            print("\n--->", val.upper(), "STRATEGY <---")
            print("     Target Amount:", mart_single_list[temp][val].iloc[0])
            print("     Target Position:", (mart_single_list[temp].head(1).index[0]) + 1)

            # Highest values
            print("    ", "-" * 20)
            print("     Highest Profit:", mart_single_list[temp][val].max())
            print("     Highest Profit POSITION:", (odd_df[odd_df[val] == mart_single_list[temp][val].max()].index[0]) + 1)

        if odd_df[val].min() < 0:
            print("    ", "-" * 20)
            print("     Maximum Loss:", odd_df[odd_df[val] < 0][val].min()) 
            print("     Maximum Loss POSITION:", (odd_df[odd_df[val] == odd_df[odd_df[val] < 0][val].min()].index[0]) + 1, "\n")

    print("*" * 100)


TOTAL GAMES: 646 

Highest Loss : -11100
Single Target - 1000.0
Total Target - 3000.0 

TOTAL GAMES: 38
     --------------------
     Maximum Loss: -5400
     Maximum Loss POSITION: 14 

     --------------------
     Maximum Loss: -8020
     Maximum Loss POSITION: 18 

****************************************************************************************************
Highest Loss : -11100
Single Target - 1000.0
Total Target - 3000.0 

TOTAL GAMES: 38
     --------------------
     Maximum Loss: -3700
     Maximum Loss POSITION: 5 

     --------------------
     Maximum Loss: -5000
     Maximum Loss POSITION: 5 

****************************************************************************************************
Highest Loss : -11100
Single Target - 1000.0
Total Target - 3000.0 

TOTAL GAMES: 38

---> MARTINGALE_BANKROLL STRATEGY <---
     Target Amount: 3300
     Target Position: 18
     --------------------
     Highest Profit: 5300
     Highest Profit POSITION: 24
     ----------

In [230]:
# For 3% Target
martingale_strategy = [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
single_stake_strategy = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
better_performance = ["S", "S", "S", "M", "S", "M", "S", "S", "S", "S", "S", "S", "M", "", "S", "S"]


In [421]:
odd_df.head(50)

,week,odd_position_num,odd,H_score,A_score,T_goals,Single_Bankroll,Check_single,Martingale_Bankroll,Check_Martingale
0,20.0,8.0,3.85,3.0,1.0,4.0,2850,-,1000,-
1,21.0,5.0,3.76,1.0,0.0,1.0,1850,-,1000,-
2,22.0,6.0,3.76,1.0,0.0,1.0,850,-,1000,-
3,23.0,6.0,3.88,4.0,1.0,5.0,3730,Achieved,2000,-
4,24.0,10.0,3.76,1.0,2.0,3.0,2730,-,2000,-
5,25.0,1.0,3.88,1.0,1.0,2.0,1730,-,2000,-
6,26.0,8.0,3.85,3.0,1.0,4.0,4580,Achieved,3000,Achieved
7,27.0,1.0,3.85,1.0,1.0,2.0,3580,Achieved,3000,Achieved
8,28.0,4.0,3.76,2.0,1.0,3.0,2580,-,3000,Achieved
9,29.0,6.0,3.85,1.0,0.0,1.0,1580,-,3000,Achieved


In [392]:
odd_df[50: 100]

,week,odd_position_num,odd,H_score,A_score,T_goals,Single_Bankroll,Check_single,Martingale_Bankroll,Check_Martingale
50,23.0,2.0,3.83,0.0,2.0,2.0,-520,-,-1400,-
51,24.0,6.0,3.85,2.0,3.0,5.0,-520,-,-1400,-
52,25.0,5.0,3.76,4.0,1.0,5.0,2240,-,-400,-
53,26.0,7.0,3.85,0.0,2.0,2.0,1240,-,-400,-
54,27.0,4.0,3.48,4.0,2.0,6.0,3720,Achieved,600,-
55,28.0,9.0,3.85,2.0,2.0,4.0,6570,Achieved,1600,-
56,29.0,3.0,3.43,2.0,1.0,3.0,5570,Achieved,1600,-
57,30.0,1.0,3.45,1.0,0.0,1.0,4570,Achieved,1600,-
58,31.0,6.0,3.83,0.0,2.0,2.0,3570,Achieved,1600,-
59,32.0,5.0,3.43,1.0,3.0,4.0,6000,Achieved,2600,-


In [322]:
odd_df[100: 150]

,week,odd_position_num,odd,H_score,A_score,T_goals,Single_Bankroll,Check_single,Martingale_Bankroll,Check_Martingale
100,7.0,1.0,3.85,1.0,0.0,1.0,4640,Achieved,-2200,-
101,8.0,1.0,3.40,2.0,1.0,3.0,4640,Achieved,-2200,-
102,9.0,8.0,3.88,1.0,0.0,1.0,4640,Achieved,-2200,-
103,10.0,9.0,3.85,0.0,2.0,2.0,4640,Achieved,-2200,-
104,11.0,8.0,3.85,2.0,1.0,3.0,4640,Achieved,-2200,-
105,12.0,7.0,3.58,0.0,0.0,0.0,4640,Achieved,-2200,-
106,13.0,5.0,3.76,1.0,0.0,1.0,4640,Achieved,-2200,-
107,14.0,10.0,3.88,0.0,0.0,0.0,4640,Achieved,-2200,-
108,15.0,3.0,3.85,0.0,3.0,3.0,4640,Achieved,-2200,-
109,16.0,4.0,3.83,3.0,1.0,4.0,4640,Achieved,-2200,-


In [246]:
odd_df[150: 200]

,week,odd_position_num,odd,H_score,A_score,T_goals,Single_Bankroll,Check_single,Martingale_Bankroll,Check_Martingale
150,37,10,3.40,2,3,5,5600,Achieved,5400,Achieved
151,38,10,3.43,3,2,5,8030,Achieved,6400,Achieved
152,1,10,3.08,0,2,2,7030,Achieved,6400,Achieved
153,2,4,3.85,0,0,0,6030,Achieved,6400,Achieved
154,3,1,3.76,1,5,6,8790,Achieved,7400,Achieved
155,4,5,3.76,3,0,3,7790,Achieved,7400,Achieved
156,5,7,3.29,1,1,2,6790,Achieved,7400,Achieved
157,6,2,3.53,2,0,2,5790,Achieved,7400,Achieved
158,7,6,3.83,2,4,6,8620,Achieved,8400,Achieved
159,8,6,3.85,1,0,1,7620,Achieved,8400,Achieved


In [176]:

count = 1
martingale_list = []
for x in odd_df["T_goals"]:
    if x > 3:
        # print(count)
        martingale_list.append(count)
        count = 1             
    else:
        count += 1

print(martingale_list)
print(sorted(martingale_list, reverse=True)[:50])
print("Hightest Martingale Level:", max(martingale_list))


single_target = 1000
loss_amount = 3700

win_length = sorted(martingale_list, reverse=True)
total_diff = 0
for x in win_length:
    if x > 5:
        total_diff += 1
    else:
        break
print("\nTotal Played:", len(martingale_list))
print("Total Lost:", total_diff)
won = len(martingale_list) - total_diff
print("Total Won:", won)
print("Profit/Loss Amount:", (won * single_target) - (total_diff * loss_amount))

[1, 4, 2, 4, 10, 1, 11, 2, 7, 3, 2, 6, 8, 6, 1, 8, 1, 3, 3, 1, 2, 2, 2, 15, 1, 3, 3, 3, 6, 1, 4, 1, 1, 1, 3, 4, 3, 2, 1, 2, 3, 4, 6, 1, 2, 1, 4, 9, 2, 2, 2, 1, 1, 2, 3, 9, 31, 4, 1, 4, 3, 3, 9, 4, 14, 4, 1, 1, 1, 2, 6, 3]
[31, 15, 14, 11, 10, 9, 9, 9, 8, 8, 7, 6, 6, 6, 6, 6, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
Hightest Martingale Level: 31

Total Played: 72
Total Lost: 16
Total Won: 56
Profit/Loss Amount: -3200


## Single Stake Strategy

In [154]:
targets = odd_df[odd_df["Check_single"] == "Achieved"]
print("Total Games:", len(odd_df))
print("Target Position:", (targets.head(1).index[0]) + 1)

# Highest Profit
print("Highest Profit:", targets["Single_Bankroll"].max())
print("Highest Profit POSITION:", (odd_df[odd_df["Single_Bankroll"] == targets["Single_Bankroll"].max()].index[0]) + 1)

Total Games: 27
Target Position: 26
Highest Profit: 6820
Highest Profit POSITION: 26


In [155]:
targets

,week,odd_position_num,odd,H_score,A_score,T_goals,Single_Bankroll,Check_single,Martingale_Bankroll,Check_Martingale
25,8,8,3.85,4,2,6,6820,Achieved,-18500,-
26,9,9,3.42,0,1,1,5820,Achieved,-18500,-
